<a href="https://colab.research.google.com/github/shivammehta007/Information-Retrieval/blob/master/MLinRanking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of ML in Ranking

# Application of ML in Ranking of Search Engines

## Downloading Data From Kaggle
#### Might Take a while

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Download Data
!pip install kaggle
import json
import os

with open('/content/drive/My Drive/kaggle.json') as kc:
    kaggle_config = json.load(kc)

os.environ['KAGGLE_USERNAME'] = kaggle_config['username'] 
os.environ['KAGGLE_KEY'] = kaggle_config['key'] 

!kaggle competitions download -c text-relevance-competition-ir-2-itmo-fall-2019
!unzip docs.tsv.zip
!rm docs.tsv.zip

sample.submission.txt: Skipping, found more recently modified local copy (use --force to force download)
100% 1.17G/1.17G [00:05<00:00, 255MB/s]
100% 1.17G/1.17G [00:05<00:00, 240MB/s]
queries.numerate.txt: Skipping, found more recently modified local copy (use --force to force download)
Archive:  docs.tsv.zip
replace docs.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Code

In [19]:
!pip install -U tqdm
!pip install rank_bm25
!pip install keyw
import os
import gzip
import nltk
import random
import pickle
import pandas as pd
import keyw
import logging
import itertools
import re
import time

import numpy as np
import pandas as pd

from rank_bm25 import BM25Okapi

from nltk.corpus import stopwords
nltk.download("stopwords")

from string import punctuation
from collections import defaultdict
from collections import OrderedDict
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


logging.basicConfig(level=logging.DEBUG)
logging.debug('Test Logger')

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.41.1)


DEBUG:root:Test Logger


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
indexing_data_location = '/content/docs.tsv'
queries_list = '/content/queries.numerate.txt'

### Building Inverted Index

#### Inverted Indexer Class

In [0]:
## Skip This for now its not implemented yet
# # Spelling manager
# class Trie:
#     """
#     Trie

#     Trie is a treelike datastructure used to predict suggestion here is a
#     simple implementation of it with add and search functionality.

#     It has a root that is the head or the Trie and a node_count to count
#     total number of nodes currently in Trie
#     """

#     class _Node:
#         """
#         Node

#         This is how a trie node looks like it has a hashmap of characters
#         with an end indicating weather this is the end of word or not.
#         One additional field that I added is the frequency count just for
#         furthur probabilistic calculations if required.
#         """

#         def __init__(self, end=False):
#             self.characters = {}
#             self.frequency = 0
#             self.end = end

#     def __init__(self):
#         self.root = self._Node()
#         self.node_count = 1

#     def add_string(self, string):
#         """
#         Adds a string to the trie
#         Parameters:
#         string: String
#         """
#         node = self.root
#         for c in string:
#             if c not in node.characters:
#                 node.characters[c] = self._Node()
#                 self.node_count += 1

#             node = node.characters[c]
#             node.frequency += 1

#         node.end = True

#     def search_word(self, string):
#         """
#         Searches for a word in the trie
#         Parameters:
#         string: String
#         """
#         node = self.root
#         for c in string:
#             if c not in node.characters:
#                 return False
#             node = node.characters[c]

#         if node.end:
#             return True

#         return False

In [0]:
class InvertedIndexer:
    """
    This class makes inverted index
    """

    def __init__(self, filename=False):
        self.filename = filename
        self.stemmer_ru = nltk.SnowballStemmer("russian")
        self.stopwords = set(stopwords.words("russian")) | set(stopwords.words("english"))
        # self.trie = Trie()
        self.punctuation = punctuation # from string import punctuation
        if filename:
            self.inverted_index = self._build_index(self.filename)
        else:
            self.inverted_index = defaultdict(set)

    def preprocess(self, sentence):
        """
        Method to remove stop words and punctuations return tokens
        """
        NONTEXT = re.compile('[^0-9 a-z#+_а-яё]')

        sentence = sentence.lower()
        sentence = sentence.translate(str.maketrans('', '', punctuation))
        sentence = re.sub(NONTEXT,'',sentence)

        # Heavy Operation Taking lot of time will move it outside
        # tokens = [self.stemmer_ru.stem(word) for word in sentence.split()]

        tokens = [token for token in sentence.split() if token not in self.stopwords]
        # for t in tokens:
        #     self.trie.add_string(t)

        return tokens

    def stem_keys(self, inverted_index):
        """
        Called after index is built to stem all the keys and normalize them
        """
        logging.debug('Indexing Complete will not Stem keys and remap indexes')
        temp_dict = defaultdict(set)
        i = 0
        for word in tqdm(inverted_index):

            stemmed_key = self.stemmer_ru.stem(word)
            temp_dict[stemmed_key].update(inverted_index[word])
            inverted_index[word] = None

        inverted_index = temp_dict
        logging.debug('Done Stemmping Indexes')
        return inverted_index

    def _build_index(self, indexing_data_location):
        """
        This method builds the inverted index and returns the invrted index dictionary
        """
        inverted_index = defaultdict(set)
        with open(indexing_data_location, "rb") as f:
            for line in tqdm(f):
                line = line.decode().split('\t')
                file_number = line[0]
                subject = line[1]
                text = line[2]
                line = subject + ' ' + text

                for word in self.preprocess(line):
                        inverted_index[word].add(int(file_number))

        # Intermediate Save
        # with open('/content/intermediate_inverted_index.pickle', mode='wb') as f:
        #     pickle.dump(inverted_index, f)
        # Collab crashed don't do that 

        inverted_index = self.stem_keys(inverted_index)

        return inverted_index

    def save(self, filename_to_save):
        """
        Save method to save the inverted indexes
        """
        with open(filename_to_save, mode='wb') as f:
            pickle.dump(self.inverted_index, f)
        
        # with open(filename_to_save + '_trie', mode='wb') as f:
        #     pickle.dump(self.trie, f)

    def load(self, filelocation_to_load):
        """
        Load method to load the inverted indexes
        """
        with open(filelocation_to_load, mode='rb') as f:
            self.inverted_index = pickle.load(f)

        # with open(filelocation_to_load + '_trie', mode='rb') as f:
        #     self.trie = pickle.load(f)


#### SolutionPredictor Class

In [0]:
class SolutionPredictor:
    """
    This classes uses object of Hw1SolutionIndexer
    to make boolean search
    """
    def __init__(self, indexer):
        """
        indexer : object of class Hw1SolutionIndexer
        """
        self.indexer = indexer


    def find_docs(self, query):
        """
        This method provides booleaen search
        query : string with text of query
        Returns Python set with documents which contain query words
        Should return maximum 100 docs
        """

        query = keyw.engrus(query)
        tokens = self.indexer.preprocess(query)
        tokens = [self.indexer.stemmer_ru.stem(word) for word in tokens]
        docs_list = set()
        for word in tokens:
            if word in self.indexer.inverted_index:
                if len(docs_list) > 0:
                    docs_list.intersection_update(self.indexer.inverted_index[word]) # changed from intersection_update
                else:
                    docs_list.update(self.indexer.inverted_index[word])
        
        # Handling case when no set is returned by intersection
        # if len(docs_list) < 10:
        #     for word in tokens:
        #         docs_list.update(self.indexer.inverted_index[word]) # changed from intersection_update

        return docs_list

#### Generate Index 
Run Only when Index is not generated Otherwise use the Loading code block

In [0]:
logging.debug('Index is creating...')
start = time.time()
new_index = InvertedIndexer(indexing_data_location) 
end = time.time()
logging.debug('Index has been created and saved as inverted_index.pickle in {:.4f}s'.format(end-start))

DEBUG:root:Index is creating...


DEBUG:root:Indexing Complete will not Stem keys and remap indexes


DEBUG:root:Done Stemmping Indexes
DEBUG:root:Index has been created and saved as inverted_index.pickle in 1007.3685s


In [0]:
len(new_index.inverted_index)

2858862

In [0]:
new_index.save('inverted_index.pickle')

In [0]:
!cp inverted_index.pickle '/content/drive/My Drive/Homeworks/InformationRetrieval/inverted_index_4.pickle'

#### Load Index
Run when Index is not generated

In [0]:
index_location = '/content/drive/My Drive/Homeworks/InformationRetrieval/inverted_index_all_rus.pickle'

In [8]:
logging.debug('Loading Index...')
start = time.time()
new_index = InvertedIndexer()
new_index.load(index_location)
end = time.time()
logging.debug('Index has been loaded from inverted_index.pickle in {:.4f}s'.format(end-start))

DEBUG:root:Loading Index...
DEBUG:root:Index has been loaded from inverted_index.pickle in 76.2058s


In [9]:
len(new_index.inverted_index)

2820826

#### Convert keys to all Russian

In [27]:
i = 0 
for key in new_index.inverted_index:
    print(new_index.inverted_index[key])
    i += 1
    if i == 5:
        break

{43011, 493573, 92166, 186375, 233480, 159756, 368654, 145423, 274449, 131090, 71699, 86036, 47126, 188438, 380952, 403481, 24, 90137, 368666, 188443, 159769, 350239, 403483, 532510, 479268, 40998, 39, 368681, 479276, 368685, 221230, 88111, 495665, 557109, 550971, 450625, 41030, 233548, 323663, 489553, 381010, 362581, 159830, 405589, 192602, 438362, 233565, 495717, 510055, 528488, 313451, 536683, 397308, 452719, 538739, 348277, 22646, 303223, 376952, 528502, 303227, 303228, 442493, 276606, 348289, 204931, 313475, 149639, 538766, 221330, 489618, 489620, 209043, 256150, 221336, 254104, 159901, 362658, 362662, 448678, 28840, 112808, 448682, 448683, 489644, 389293, 495783, 448687, 20656, 479407, 254130, 323762, 327858, 364725, 14518, 489651, 61624, 440508, 516286, 577727, 18626, 35010, 366786, 579779, 278726, 364743, 307402, 577738, 112846, 100559, 577743, 135378, 440531, 168148, 149717, 90326, 149719, 216, 92376, 172246, 110811, 220, 235732, 538840, 164064, 32994, 78053, 221414, 209130, 1

In [0]:
temp_dict = defaultdict(set)
i = 0
for word in new_index.inverted_index:
    i += 1
    if i % 100000 == 0:
        print(i)
    rus_key = keyw.engrus(word)

    temp_dict[rus_key].update(new_index.inverted_index[word])
    new_index.inverted_index[word] = None

new_index.inverted_index = temp_dict
temp_dict = None
del temp_dict


In [29]:
len(new_index.inverted_index)

4756065

In [0]:
new_index.save('inverted_index_all_rus_no_stem.pickle')

In [0]:
!cp inverted_index_all_rus_no_stem.pickle '/content/drive/My Drive/Homeworks/InformationRetrieval/inverted_index_all_rus_no_stem.pickle'

##### Testing Block

In [0]:
test_sentence = 'бандиты боятся ли ментов'

In [0]:
boolean_model = SolutionPredictor(new_index)

In [28]:
start = time.time()
value_present = boolean_model.find_docs(test_sentence)
print(value_present)
logging.debug('Search Time: {}'.format(time.time() - start))

DEBUG:root:Search Time: 0.0024552345275878906


{59393, 122885, 92166, 370697, 262166, 477206, 575512, 110616, 145432, 432154, 432166, 247847, 106540, 557108, 421941, 12351, 522305, 190534, 221260, 198735, 284753, 579665, 364632, 505945, 364635, 178274, 313443, 313451, 419951, 489583, 75893, 22646, 57466, 489595, 180351, 106626, 108677, 88199, 221320, 112780, 489618, 489620, 485525, 489622, 221335, 221336, 383136, 364705, 362658, 362662, 204967, 489639, 112810, 245933, 20656, 489651, 577718, 403642, 383163, 61629, 577727, 14529, 403650, 579780, 563400, 577738, 413900, 14540, 577743, 520399, 135378, 379091, 362708, 340183, 92376, 383196, 442590, 221414, 489710, 299250, 155891, 510199, 311545, 311553, 430350, 495888, 92435, 430358, 364825, 416028, 174377, 174380, 10544, 284977, 284978, 452915, 414004, 414006, 311610, 284990, 284993, 381257, 414037, 383317, 108887, 293207, 389467, 98656, 414049, 414054, 346471, 164202, 383339, 328049, 358783, 489860, 108939, 383372, 213389, 311694, 98709, 72093, 489886, 311712, 266657, 184738, 489897, 

In [0]:
data = database[database['id'].isin(list(value_present))].copy()
data.loc[:, 'content'] = data[['subject', 'content']].astype(str).apply(' '.join, axis=1)
del data['subject']
data = OrderedDict(data.set_index('id')['content'].to_dict())
for key in data:
    data[key] = ' '.join(new_index.preprocess(data[key]))



In [0]:
i = 0
for key in data:
    i += 1
    print(key, data[key])
    if i == 5:
        break

In [0]:
ids = data.keys()

In [0]:
tf_idf_test = TfidfVectorizer()
data = tf_idf_test.fit_transform(data.values())

In [0]:
query = tf_idf_test.transform([test_sentence])[0]

In [0]:
distances = cosine_similarity(query, data)

In [49]:
sorted(list(zip(ids, distances[0])), key=lambda x: x[1], reverse=True)[:10] 

[(178274, 0.13960637256481664),
 (218744, 0.09090821605679983),
 (340183, 0.08117254692310241),
 (336699, 0.05268040409627736),
 (307077, 0.031005420992367423),
 (20177, 0.02704131147882186),
 (61288, 0.026743387069791263),
 (152741, 0.024239895257922528),
 (120156, 0.02343521737850251),
 (187687, 0.02280658489060018)]

### Loading File to DataFrame
We are doing this for faster search based on index numbers

In [21]:
%%time
database = pd.read_csv(indexing_data_location, sep='\t', header=None,
                       names=['id', 'subject', 'content'],
                       dtype = {'id': int,'subject': str,
                            'content': str})

CPU times: user 55.2 s, sys: 13.3 s, total: 1min 8s
Wall time: 2min 15s


In [22]:
database.head()

,id,subject,content
0,24,БОЛЕЕТ ЗА ФУТБОЛ И ТРАХАЕТ ДОЧЬ СОФТ ОРГАНАЙЗЕ...,ГЛАВНАЯ НОВОСТИ КАРТА СОФТ ОРГАНАЙЗЕР СКАЧАТЬ ...
1,26,МОНГОЛЬСКАЯ ТХАНКА СОФТФОН,ГЛАВНАЯ НОВОСТИ КАРТА СОФТФОН GT МОНГОЛЬСКАЯ Т...
2,30,СМОТРЕТЬ ФИЛЬМЫ ОНЛАЙН БЕСПЛАТНО СМОТРЕТЬ КОНЦ...,ЗАГРУЗКА ПОЖАЛУЙСТА ПОДОЖДИТЕ ВНИМАНИЕ ОБНАРУЖ...
3,35,КАК ПРИГОТОВИТЬ ТОРТ РАФАЭЛЛО В ДОМАШНИХ УСЛОВ...,ՀԱՅԵՐԵՆ РУССКИЙ ПОЛИТИКА ВНУТРЕННИЙ ВНЕШНИЙ ЭК...
4,39,МИФЫ О ГВ BREASTFEEDING AM,TOGGLE NAVIGATION BREASTFEEDING AM ГОРЯЧАЯ ЛИН...


### Implementing BM25

In [16]:
result_docs = OrderedDict()

with open(queries_list) as queries:
    for line in tqdm(queries.readlines()):
        line = line.split('\t')
        id = int(line[0])
        query = line[1]
        value_present = boolean_model.find_docs(query)
        result_docs[id] = value_present


In [0]:
for key in result_docs:
    if len(result_docs[key]) < 10:
        print(key, result_docs[key])

In [53]:
# boolean_model = SolutionPredictor(new_index)
output=open('submission.csv', 'w')
with open(queries_list) as queries:
    output.write('QueryId,DocumentId\n')
    for line in tqdm(queries.readlines()):
        line = line.split('\t')
        id = int(line[0])
        query = line[1]
        value_present = boolean_model.find_docs(query)
        data = database[database['id'].isin(list(value_present))].copy()
        data.loc[:, 'content'] = data[['subject', 'content']].astype(str).apply(' '.join, axis=1)
        del data['subject']
        data = OrderedDict(data.set_index('id')['content'].to_dict())
        for key in data:
            data[key] = ' '.join(new_index.preprocess(data[key]))
        
        if data:
            tfidf = TfidfVectorizer()
            ids = data.keys()
            data = tfidf.fit_transform(data.values())
            query = tfidf.transform([query])[0]
            distances = cosine_similarity(query, data)
            rankings =  sorted(list(zip(ids, distances[0])), key=lambda x: x[1], reverse=True)[:10] 
            for rank in rankings:
                output.write('{},{}\n'.format(id, rank[0]))

            



        data = None
        query = None
        distances = None
        rankings = None

output.close()

In [0]:
!cp submission.csv '/content/drive/My Drive/Homeworks/InformationRetrieval/submission_tfidf.csv'

In [0]:
# boolean_model = SolutionPredictor(new_index)
output=open('submission.csv', 'w')
with open(queries_list) as queries:
    output.write('QueryId,DocumentId\n')
    for line in tqdm(queries.readlines()):
        line = line.split('\t')
        id = int(line[0])
        query = line[1]
        value_present = boolean_model.find_docs(query)
        data = database[database['id'].isin(list(value_present))].copy()
        data.loc[:, 'content'] = data[['subject', 'content']].astype(str).apply(' '.join, axis=1)
        del data['subject']
        data = OrderedDict(data.set_index('id')['content'].to_dict())
        for key in data:
            data[key] = new_index.preprocess(data[key])
        if data:
            bm25 = BM25Okapi(data.values())
            rankings =  sorted(list(zip(data.keys(), bm25.get_scores(new_index.preprocess(query)))), key=lambda x: x[1], reverse=True)[:10]
            for rank in rankings:
                output.write('{},{}\n'.format(id, rank[0]))
        
        if (not data) or len(rankings) < 10:
            rankings = []
            logging.debug('Found Some values with no query results or less than 10 {} ranking'.format(id, len(rankings)))
            print(id, query)
            # for i in range(10 - len(rankings)):
            #     output.write('{},{}\n'.format(id, random.randrange(50000)))
output.close()

DEBUG:root:Found Some values with no query results or less than 10 4420 ranking


4420 ли экспрэс



DEBUG:root:Found Some values with no query results or less than 10 6267 ranking


6267 что хначит datalova


